# SNF lab

### Task (45 minutes) 

Data: MOFA's CLL dataset

1. Load and prepare data, then compute the affinity matrices and perform SNF.
2. Assume that there are two cancer subtypes and cluster them :)
3. Plot the fused network and the clusters using networkx, or Gephi, or igraph, whatever else you prefer.

Optional:
- Compare clusters to NMF
- Use the other omics tables as well
- Compare with the results from the MOFA analysis
https://bioconductor.riken.jp/packages/3.9/bioc/vignettes/MOFA/inst/doc/MOFA_example_CLL.html

Questions:


Code:
- Lab example uses a SNF implementation in Python:
    - https://github.com/rmarkello/snfpy#usage
    - https://snfpy.readthedocs.io/en/latest/usage.html
- For R, feel free to use this package:
    - https://cran.r-project.org/web/packages/NMF/index.html

In [2]:
from snf import datasets
simdata = datasets.load_simdata()
sorted(simdata.keys())

['data', 'labels']

In [3]:
n_dtypes = len(simdata.data)
n_samp = len(simdata.labels)
print('Simdata has {} datatypes with {} samples each.'.format(n_dtypes, n_samp))

Simdata has 2 datatypes with 200 samples each.


In [10]:
simdata.data[0].shape

(200, 2)

In [11]:
data_loc = "./data/"
import pandas as pd
df_meth = pd.read_csv(data_loc + "CLL_data_Methylation.csv", index_col=0)
df_mrna = pd.read_csv(data_loc + "CLL_data_mRNA.csv", index_col=0)

# drop nans by column
df_mrna = df_mrna.dropna(axis='columns')
df_meth = df_meth.dropna(axis='columns')

df_mrna = df_mrna.T
df_meth = df_meth.T

for c in df_meth.columns:
    mask = df_meth[c] < 0
    df_meth[c + '_n'] = df_meth[c].mask(mask)
    df_meth[c + '_p'] = - df_meth[c].mask(~mask)
    df_meth = df_meth.drop([c], axis=1)
df_meth = df_meth.fillna(0)

X = pd.concat([df_mrna.T, df_meth.T])
X = X.dropna(axis='columns')
print(X.shape)

(13496, 135)


In [14]:
df_mrna.T.shape[0]

(5000, 136)

In [29]:
X1 = X.iloc[5000:, :].T
X2 = X.iloc[:5000, :].T

In [30]:
print(X1.values.shape, X2.values.shape)

(135, 8496) (135, 5000)


In [31]:
from snf import compute
affinities = compute.make_affinity([X1.values, X2.values], metric='euclidean', K=20, mu=0.5)

In [34]:
affinities[0].shape

(135, 135)

In [35]:
fused = compute.snf(affinities, K=20)
first, second = compute.get_n_clusters(fused)
first, second

(2, 4)

In [40]:
import numpy as np
all_labels = np.array([1 for i in range(135)] + [2 for i in range(135)])

In [45]:
from sklearn import cluster
fused_labels = cluster.spectral_clustering(fused, n_clusters=first)
#labels = [all_labels, fused_labels]
labels = [fused_labels]
for arr in affinities:
    labels += [cluster.spectral_clustering(arr, n_clusters=first)]

In [46]:
fused_labels

array([0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 0], dtype=int32)

In [47]:
labels

[array([0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0,
        1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
        1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1,
        1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
        0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0,
        1, 1, 0], dtype=int32),
 array([0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0,
        1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
        1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
        0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0,
        1, 1, 0], dtype=int32),
 array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [48]:
from snf import metrics
nmi = metrics.nmi(labels)
print(nmi)

[[1.         0.68746257 0.01308389]
 [0.68746257 1.         0.00350462]
 [0.01308389 0.00350462 1.        ]]


In [50]:
import numpy as np
np.fill_diagonal(fused, 0)
sil = metrics.silhouette_score(fused, fused_labels)
print('Silhouette score for the fused matrix is: {:.2f}'.format(sil))

Silhouette score for the fused matrix is: 0.24
